In [ ]:
!apt-get install openjdk-8-jdk-headless

In [ ]:
!wget https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.2.1-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
findspark.find()

'/content/spark-3.2.1-bin-hadoop2.7'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("DataCleaningApp")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

In [ ]:
#loadining datasets
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import year, month, date_format ,hour , dayofweek
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

mention_df = spark.read.format("csv").option("header", "true").load("/content/sample_data/chat_mention_team_chat.csv")

leave_df = spark.read.format("csv").option("header", "true").load("/content/sample_data/chat_leave_team_chat.csv")

join_df = spark.read.format("csv").option("header", "true").load("/content/sample_data/chat_join_team_chat.csv")

repond_df = spark.read.format("csv").option("header", "true").load("/content/sample_data/chat_respond_team_chat.csv")


Join Chat

In [ ]:
# gettind data for graph of highest join
from pyspark.sql.functions import count, desc

top_10_JoinTeams = join_df.groupBy("teamchat_session_id") \
                .agg(count("user_id").alias("total_joins")) \
                .orderBy(desc("total_joins")) \
                .limit(10)

top_10_JoinTeams.show()

+-------------------+-----------+
|teamchat_session_id|total_joins|
+-------------------+-----------+
|               6792|        100|
|               6783|         91|
|               6925|         87|
|               6850|         86|
|               6791|         81|
|               6780|         76|
|               6809|         72|
|               6819|         70|
|               7212|         67|
|               6889|         67|
+-------------------+-----------+



In [ ]:
from pyspark.sql.functions import count

# Group the data by team_chat_session_id and have smallest number of occurrences
team_counts = join_df.groupBy("teamchat_session_id").agg(count("*").alias("count"))

sorted_team_counts = team_counts.orderBy("count")

teams_count = sorted_team_counts.limit(10)

teams_count.show()


+-------------------+-----+
|teamchat_session_id|count|
+-------------------+-----+
|              19403|    1|
|              58852|    1|
|              11602|    1|
|              62531|    1|
|              53235|    1|
|              33007|    2|
|              34969|    4|
|              30251|    4|
|              48730|    4|
|              53495|    4|
+-------------------+-----+



LEAVE CHAT

In [ ]:
# gettind data for graph of highest Leaves
from pyspark.sql.functions import count, desc

top_10_LeaveTeams = leave_df.groupBy("teamchat_session_id") \
                .agg(count("user_id").alias("total_Leaves")) \
                .orderBy(desc("total_Leaves")) \
                .limit(10)

top_10_LeaveTeams.show()

+-------------------+------------+
|teamchat_session_id|total_Leaves|
+-------------------+------------+
|               6792|          85|
|               6783|          77|
|               6850|          73|
|               6925|          73|
|               6791|          68|
|               6780|          65|
|               6809|          63|
|               6789|          59|
|               6819|          59|
|               7212|          58|
+-------------------+------------+



In [ ]:
from pyspark.sql.functions import count

# Group the data by team_chat_session_id and have smallest number of occurrences
team_counts = leave_df.groupBy("teamchat_session_id").agg(count("*").alias("count"))

sorted_team_counts = team_counts.orderBy("count")

teams_count = sorted_team_counts.limit(10)

teams_count.show()


+-------------------+-----+
|teamchat_session_id|count|
+-------------------+-----+
|              26541|    2|
|              48448|    2|
|              30251|    3|
|              22785|    3|
|              48730|    3|
|              34969|    3|
|              53495|    3|
|              26437|    3|
|              49085|    3|
|              40690|    4|
+-------------------+-----+



Mentioned 

In [ ]:
# gettind data for graph of highest mention
from pyspark.sql.functions import count, desc

top_10_MentinedUser= mention_df.groupBy("user_id") \
                .agg(count("chat_item").alias("total_mentined")) \
                .orderBy(desc("total_mentined")) \
                .limit(10)

top_10_MentinedUser.show()

+-------+--------------+
|user_id|total_mentined|
+-------+--------------+
|    131|            53|
|    621|            47|
|   1204|            47|
|   1428|            46|
|   1506|            46|
|   1482|            42|
|    283|            42|
|   1450|            42|
|    674|            42|
|   1035|            41|
+-------+--------------+



Neo4j Codes

In [ ]:
------RESPOND TO-------------------------------------

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/wasiqamn/Big-Data/47f5b9611684bd0201d669fd047064576b1a5e01/chat_respond_team_chat.csv' AS row
MERGE (c1:ChatItem {id: row.chatid1})
MERGE (c2:ChatItem {id: row.chatid2})
CREATE (c2)-[:RESPONDS {timestamp: row.date}]->(c1)

MATCH (c2:ChatItem)-[r:RESPONDS]->(c1:ChatItem)
RETURN c2, r, c1


MATCH path = (c2:ChatItem)-[:RESPONDS*1..]->(c1:ChatItem)
WITH c1, c2, length(path) AS response_length
ORDER BY response_length DESC
LIMIT 1
RETURN c1.id AS Chat1, c2.id AS Chat2, response_length AS Longest_Response;

--------------------JOIN--------------------------
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/wasiqamn/Big-Data/47f5b9611684bd0201d669fd047064576b1a5e01/chat_join_team_chat.csv' AS row
MERGE (u:User {id: row.user_id})
MERGE (t:TeamChatSession {id: row.teamchat_session_id})
CREATE (u)-[:Joins {timestamp: row.date}]->(t)

MATCH (u:User)-[:Joins]->(t:TeamChatSession)
RETURN u, t
----------------------LEAVE --------------------
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/wasiqamn/Big-Data/47f5b9611684bd0201d669fd047064576b1a5e01/chat_leave_team_chat.csv' AS row
MATCH (u:User {id: row.user_id})
MATCH (t:TeamChatSession {id: row.teamchat_session_id})
MERGE (u)-[:Leaves {timestamp: row.date}]->(t)


MATCH (u:User)-[r:Leaves]->(t:TeamChatSession)
RETURN u, t
-----------------------------MENTIONED-------------
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/wasiqamn/Big-Data/47f5b9611684bd0201d669fd047064576b1a5e01/chat_mention_team_chat.csv' AS row
MATCH (c:ChatItem {id: row.chat_item})
MATCH (u:User {id: row.user_id})
CREATE (c)-[:Mentioned {timestamp:row.date}]->(u)


# respond and mention
match path = (u:User) - [:Mentioned] - (c:ChatItem) - [:RESPONDS] -() return path 

match path = (u:User {id: 131}) - [:Mentioned] - (c:ChatItem) - [:RESPONDS] -() return path 

# joins or leaves
match path = (u:User) - [:Joins] - (t:TeamChatSession) - [:Leaves] -() return path 